## word2vec 개선
- 계산 자원 문제 : embedding 계층(입력층 계산 낭비 감소 효과), negative sampling 손실 함수(은닉층 이후 계산 낭비 감소 효과)

In [1]:
# embedding 계층 구현

import numpy as np
W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
W[2]

array([6, 7, 8])

In [3]:
idx = np.array([1,0,3,0]) #원하는 행 번호 입력
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [4]:
# embedding 계층의 forward() 메서드 구현

class Embedding:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

In [6]:
# backward() 메서드 구현

def backward(self, dout):
    dW, = self.grads
    dW[...] = 0 #배열의 모든 요소를 0으로 초기화

    for i, word_id in enumerate(self.idx):
        dW[self.idx] += dout #중복된 인덱스는 기울기를 더함으로써 누적되도록 = 여러번 등장했다는 의미
    return None

In [7]:
#dot 연산(내적) - 정답에 해당하는 열벡터와 은닉층 뉴런의 내적

class EmbeddingDot:
    def __init__(self,W):
        self.embed = Embedding(W)
        self.params = self.embed.parmas
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

